In [ ]:
from google.colab import drive
import os
import pandas as pd
from tqdm import tqdm
import time
import os
import json
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
news_path= '/content/drive/My Drive/NCKH/news_data/03_primary/total_news.parquet'
original_news_df = pd.read_parquet(news_path)

In [ ]:
len(original_news_df)

24338

In [ ]:
original_news_df.head()

,author,content,datetime,source,summary,title,url,date,equity
0,Jayson Derrick,WeWork is postponing its initial public offeri...,2019-09-17 17:26:14,benzinga,WeWork is postponing its initial public offeri...,Analyst On WeWork: High Growth Equals High Cas...,https://www.benzinga.com/analyst-ratings/analy...,2019-09-18 17:26:14,MSFT
1,Shanthi Rexaline,Microsoft Corporation (NASDAQ: MSFT ) reported...,2020-04-30 15:47:08,benzinga,Microsoft Corporation (NASDAQ: MSFT) reported ...,Microsoft Analysts On Redmond's Q3 Beat: 'A Co...,https://www.benzinga.com/analyst-ratings/analy...,2020-04-30 15:47:08,MSFT
2,Tanzeel Akhtar,Editor's note: This story has been updated to ...,2020-05-13 16:32:36,benzinga,Editor&#39;s note: This story has been updated...,What To Know About The Conflicting AMC Enterta...,https://www.benzinga.com/m-a/20/05/16014823/wh...,2020-05-14 16:32:36,AMZN
3,Chris Katje,Despite a wealth of 162.8 billion and title of...,2021-06-25 16:30:26,benzinga,Despite a wealth of $162.8 billion and title o...,Did Billionaire Elon Musk Sell All His Mansion...,https://www.benzinga.com/news/21/06/21714362/d...,2021-06-26 16:30:26,TSLA
4,Phil Hall,Elon Musk has taken his last remaining house o...,2021-09-20 15:52:05,benzinga,Elon Musk has taken his “last remaining house”...,Elon Musk Fails To Get Buyer For $37.5M Bay Ar...,https://www.benzinga.com/news/21/09/23013145/e...,2021-09-20 15:52:05,TSLA


In [ ]:
pip install requests

In [ ]:
# Take next 3000 rows and reset the index
news_df = original_news_df[16000:].copy().reset_index(drop=True)
news_df["hf_summary"] = None  # new column for generated summary

In [ ]:
len(news_df)

8338

In [ ]:
news_df.head()

,author,content,datetime,source,summary,title,url,date,equity,hf_summary
0,Benzinga Neuro,The stock market saw a positive trend on Tuesd...,2024-11-27 01:53:37,benzinga,"AMZN, CRWD, ADSK, DELL, TSLA were among the to...","Amazon, Crowdstrike, Autodesk, Dell, And Tesla...",https://www.benzinga.com/24/11/42205689/unveil...,2024-11-27 01:53:37,TSLA,None
1,Benzinga Neuro,"On Tuesday, Cathie Wood -led Ark Invest execut...",2024-11-27 02:35:18,benzinga,"On Tuesday, Nov. 26, 2024, Ark Invest, led by ...",Cathie Wood's Latest Tesla Dump: Ark Sells $18...,https://www.benzinga.com/24/11/42205945/ark-in...,2024-11-27 02:35:18,TSLA,None
2,Ananya Gairola,ChatGPT -maker OpenAI's unreleased Sora text-t...,2024-11-27 03:19:23,benzinga,OpenAI&#39;s unreleased Sora text-to-video AI ...,OpenAI's Sora Video Model Leak Exposes Deep Ri...,https://www.benzinga.com/24/11/42206104/openai...,2024-11-27 03:19:23,MSFT,None
3,Kaustubh Bagalkote,In a provocative discourse on modern military ...,2024-11-27 04:01:23,benzinga,In a provocative discourse on modern military ...,Amid Elon Musk's Push For Drones And AI For Co...,https://www.benzinga.com/markets/equities/24/1...,2024-11-27 04:01:23,TSLA,None
4,Rounak Jain,Tesla Inc. (NASDAQ: TSLA ) on Tuesday revamped...,2024-11-27 06:24:52,benzinga,Tesla on Tuesday revamped its referral program...,"Tesla Sweetens Referral Program, Allowing Cust...",https://www.benzinga.com/markets/24/11/4220718...,2024-11-27 06:24:52,TSLA,None


In [ ]:
HF_TOKEN = "input your token here"  # 🔒 Replace with your token
ENDPOINT_URL = "https://ga02vww3k1zjo6gk.us-east-1.aws.endpoints.huggingface.cloud"

HEADERS = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

SAVE_PATH = "/content/drive/My Drive/NCKH/news_data/generated_summary_checkpoint_3.parquet"


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import date
# ==== 3. Investor-Tailored Summarization Prompt ====
# Kept the original function for compatibility if needed elsewhere
def format_prompt_for_investor(article_text, symbol):
  return f"""
for the stock {symbol}, summarize the following news article in no more than 140 words. Focus on key factual events, quantitative figures (e.g., earnings, percentage changes, dates), material risks, relevant corporate actions, regulatory developments, or market sentiment that could impact {symbol}'s stock price. Ensure all information is directly from the article and accurately reported, especially numerical figures. Avoid speculation, assumptions, or vague generalizations. Do not include opinions or inferred conclusions not directly supported by the source text.

Article:
{article_text}

Summary:
"""

# ==== 4. Summarization function with retry and flexible prompt formatting ====
def summarize_text(idx, text, symbol, prompt_formatter, max_retries=5, wait_time=10):
    full_prompt = prompt_formatter(text, symbol)
    data = {
        "inputs": full_prompt,
        "parameters": {
            "max_new_tokens": 220,
            "do_sample": False,
            "temperature": 0.5,
            # "top_p": 0.9
        }
    }

    for attempt in range(max_retries):
        try:
            response = requests.post(ENDPOINT_URL, headers=HEADERS, json=data, timeout=60)
            if response.status_code == 200:
                result = response.json()
                if isinstance(result, list):
                    text = result[0]["generated_text"]
                else:
                    text = result["generated_text"]
                return idx, text
            elif response.status_code in [429, 502, 503]:
                print(f"⚠️ Retry {attempt+1} for index {idx} - status {response.status_code}")
            else:
                print(f"❌ Error {response.status_code}: {response.text}")
        except Exception as e:
            print(f"🚨 Exception at index {idx}: {e}")
        time.sleep(wait_time)

    return idx, None  # if all retries fail

# ==== 5. Threaded Summarization with Progress and Checkpoints ====
# This part will need to be updated to pass the desired prompt_formatter function
# when you decide to run the full summarization.

In [ ]:
# Assuming summarize_text function is already defined and accepts prompt_formatter
# Assuming format_prompt_for_investor_chat and HF_TOKEN, ENDPOINT_URL, HEADERS are defined
# Assuming news_df is loaded and has 'content', 'equity', and 'hf_summary' columns

batch_size = 10 # Adjust based on your endpoint's capacity and rate limits
total_records = len(news_df)
SAVE_PATH = "/content/drive/My Drive/NCKH/news_data/generated_summary_checkpoint_3.parquet" # Make sure this path is correct

# Find the starting point if a checkpoint exists
start_index = 0
if os.path.exists(SAVE_PATH):
    checkpoint_df = pd.read_parquet(SAVE_PATH)
    # Assuming the checkpoint file has the same structure and order as the original df
    # and we are appending to it. If not, more sophisticated merging is needed.
    if not checkpoint_df.empty:
      # Find the first row in news_df that is NOT in the checkpoint_df by index
      # This assumes news_df's index is unique and corresponds to the original
      # file's row number before slicing and resetting index.
      # Since we are slicing and resetting index, a simpler approach is to
      # find the last non-null 'hf_summary' entry in the current news_df
      last_processed_index = news_df['hf_summary'].last_valid_index()
      if last_processed_index is not None:
          start_index = last_processed_index + 1
          print(f"Resuming from index {start_index}")


def summarize_batch(batch_df, prompt_formatter):
    with ThreadPoolExecutor(max_workers=batch_size) as executor:
        futures = {executor.submit(summarize_text, row.Index, row.content, row.equity, prompt_formatter): row.Index for row in batch_df.itertuples()}
        results = []
        for future in as_completed(futures):
            index = futures[future]
            try:
                results.append(future.result())
            except Exception as exc:
                print(f'Index {index} generated an exception: {exc}')
                results.append((index, None)) # Append None for failed summaries
        return results

print(f"Starting summarization from index {start_index} out of {total_records}")

# Process in batches
for i in tqdm(range(start_index, total_records, batch_size), desc="Summarizing"):
    batch_end = min(i + batch_size, total_records)
    batch_df = news_df.iloc[i:batch_end]

    # Use format_prompt_for_investor for this batch summarization
    batch_results = summarize_batch(batch_df, format_prompt_for_investor)

    # Update the DataFrame with results
    for idx, summary in batch_results:
        news_df.loc[idx, "hf_summary"] = summary

    # Save checkpoint
    # Save the entire DataFrame to include already processed parts and the new batch
    news_df.to_parquet(SAVE_PATH, index=False)


print(f"Done! All summaries saved with batch-size={batch_size} threading.")

Starting summarization from index 0 out of 8338


Summarizing:   0%|          | 0/834 [00:00<?, ?it/s]

⚠️ Retry 1 for index 1 - status 503
⚠️ Retry 1 for index 0 - status 503
⚠️ Retry 1 for index 3 - status 503
⚠️ Retry 1 for index 2 - status 503
⚠️ Retry 1 for index 6 - status 503
⚠️ Retry 1 for index 5 - status 503
⚠️ Retry 1 for index 7 - status 503
⚠️ Retry 1 for index 9 - status 503
⚠️ Retry 1 for index 8 - status 503
⚠️ Retry 1 for index 4 - status 503
⚠️ Retry 2 for index 1 - status 503
⚠️ Retry 2 for index 0 - status 503
⚠️ Retry 2 for index 3 - status 503
⚠️ Retry 2 for index 2 - status 503
⚠️ Retry 2 for index 6 - status 503
⚠️ Retry 2 for index 5 - status 503
⚠️ Retry 2 for index 7 - status 503
⚠️ Retry 2 for index 9 - status 503
⚠️ Retry 2 for index 8 - status 503
⚠️ Retry 2 for index 4 - status 503


Summarizing: 100%|██████████| 834/834 [1:15:05<00:00,  5.40s/it]

Done! All summaries saved with batch-size=10 threading.
